In [1]:
import os
import json
from tqdm.auto import tqdm
import bz2
# from gensim.corpora import wikicorpus

In [2]:
import xml.sax

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [3]:
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

handler._pages

[]

In [4]:
for i, line in tqdm(enumerate(open("../data/raw/viwiki-20220620-pages-articles.xml"))):
    parser.feed(line)

0it [00:00, ?it/s]

In [7]:
len(handler._pages)

2114274

In [8]:
title2page = dict([(x,y) for x,y in tqdm(handler._pages)])

  0%|          | 0/2114274 [00:00<?, ?it/s]

In [33]:
redirects = dict()
for title, page in tqdm(title2page.items(), total=len(title2page)):
    if "#redirect" in page and ":" not in title:
        redirects[title.strip()] = page.split("[[",2)[1].split("]]")[0].strip()
        print(title, "->", redirects[title])

  0%|          | 0/2114274 [00:00<?, ?it/s]

ISOC Vietnam -> Internet Society
SEA Games -> Đại hội Thể thao Đông Nam Á
Trang Cộng Đồng -> Wikipedia:Cộng đồng
TPHCM -> Thành phố Hồ Chí Minh
VIetnamese Standard Code for Information Interchange -> VISCII
Khoa học về đất -> Địa chất học
Khoa học về trái đất -> Khoa học Trái Đất
Mã nguồn mở -> Phần mềm nguồn mở
Jesus -> Giê-su
Đức Chúa Giê-xu -> Giê-su
Giêxu -> Giê-su
Đạo Phật -> Phật giáo
GFDL -> Giấy phép Tài liệu Tự do GNU
Text of the GNU Free Documentation License -> Wikipedia:Nguyên văn Giấy phép Tài liệu Tự do GNU
Sinh vật học -> Sinh học
Việt Nam Cộng Hoà -> Việt Nam Cộng hòa
Corn Toss -> Cornhole
Corntoss -> Cornhole
Vuông -> Hình vuông
Tourane -> Đà Nẵng
Phi-líp-pin -> Philippines
VPN -> Mạng riêng ảo
Virtual private network -> Mạng riêng ảo
Khoa học thiên nhiên -> Khoa học tự nhiên
Khoa học về Trái đất -> Khoa học Trái Đất
Hệ quản trị cơ sở dữ liệu -> Cơ sở dữ liệu#Hệ thống quản lý cơ sở dữ liệu
Cambodia -> Campuchia
Cyclo -> Xích lô
Hóa học phân tích -> Hóa phân tích
Bắc Cạ

In [43]:
train = json.load(open("../data/processed/zac2022_train_merged_final.json"))
entity_dict = dict()
for x in train['data']:
    if x["category"] == "FULL_ANNOTATION" and "wiki/" in x["answer"]:
        entity_dict[x["short_candidate"]] = x["answer"]
for key, val in redirects.items():
    redirects[key] = f"wiki/{val}".replace(" ","_")
entity_dict.update(redirects)

In [44]:
json.dump(entity_dict, open("../data/processed/entities.json","wt"))